[![Open in Colab](https://camo.githubusercontent.com/52feade06f2fecbf006889a904d221e6a730c194/68747470733a2f2f636f6c61622e72657365617263682e676f6f676c652e636f6d2f6173736574732f636f6c61622d62616467652e737667)](https://colab.research.google.com/github/colbrydi/Scientific_Image_Understanding/blob/master/02-Segmentation_in-class-assignment.ipynb)

# In-Class Assignment: Segmentation

<img src="http://lapix.ufsc.br/sms/results/thumb_comparison2.jpg" width="50%">

Image from: http://lapix.ufsc.br/sms/segresults.html

Today we will experiment with the color based thresholding.  You will work in groups to "segment" an image. The instructor will assign a different image and goal to each group.  As a group, figure out how to load the image, which colorspace to use and the best thresholds for segmentation.  Record your final answers at the bottom of the notebook. Your should only need to turn in one notebook per group (make sure you put all of your names at the top). 


### Agenda for today's class 

1. [Color Based Segmentation](#Color_Spaces_and_Segmentation)
1. [Practice Example](#Practice_Example)

----
<a name="Color_Spaces_and_Segmentation"></a>

# 1. Color Spaces and Segmentation
In this section we are going to explore different colorspaces for images.  Image data is generally represented as matrices of one or three dimentions.  A one dimentional image is a black and white image or grayscale image. A three dimentional image is represented as color.

In [ ]:
#The following code snip-it reads any file from the internet and saves it to your local directory.
from urllib.request import urlopen, urlretrieve
from imageio import imread, imsave
from matplotlib.pylab import plt
import numpy as np
url = 'https://research.msu.edu/wp-content/uploads/2019/11/beaumont-winter.jpg'
filename = 'boumont1.jpg'
urlretrieve(url, filename);

im = imread(filename)
plt.figure(figsize=(20,10));
plt.imshow(im)

## Grayscale Images

A grayscale image is typically one value per pixel and is generally represented as a gray value from black to white. 


In [ ]:
%matplotlib inline
import matplotlib.pylab as plt
import skimage.color as color

gray = color.rgb2gray(im)

plt.figure(figsize=(10,5));
plt.imshow(gray, cmap='gray');
plt.axis('off')
plt.colorbar();

# Binary Images
Binary images are those that have only two possible values per pixel; Zero and one, True and False, etc. We can create a binary image using logical operators such as AND (&), OR (|) and NOT (!).  The following code takes two inputs gmin, gmax to make a binary image from the above grayscale image. 

&#9989; **<font color=red>DO THIS:</font>** Play with the sliders to adjust how the image is segmented by the minimum and maximum gray value

In [ ]:
from ipywidgets import interactive,fixed

def gray_threshold(im, gmin=0.0,gmax=1.0):
    if im.ndim > 1:
        im = color.rgb2gray(im)

    b_img = ((im > gmin) & (im < gmax)); # Generate binary image
    plt.figure(figsize = (10,20));
    plt.imshow(b_img, cmap='gray', vmin=0, vmax=1);
    plt.show();
    return b_img*1;

w = interactive(gray_threshold, im=fixed(gray),gmin=(0.0,1.0), gmax=(0.0,1.0),__manual=True);
w

# Red, Green, Blue (RGB) Colorspace

Although not the most efficient way to represent color the RGB colorspace is the most common because it is based on the human eye which has three cells that are each tuned to a color spectrum as follows:

<img src="https://www.unm.edu/~toolson/human_cone_action_spectra.gif">

In [ ]:
#Simple loop to print out the red, green and blue channels
names = ['Red', 'Green', 'Blue'];
f, ax = plt.subplots(1, 4,figsize=(20,10));

for c in range(3):
    c_im = np.ubyte(np.zeros(im.shape));
    c_im[:,:,c] = im[:,:,c];
    ax[c].imshow(c_im, cmap='gray');
    ax[c].axis('off')
    ax[c].set_title(names[c])

ax[3].imshow(im);
ax[3].axis('off');

The following tool creates slider bars that you can use to play with the bounds on each of the color channels (red, green and blue).  

&#9989; **<font color=red>DO THIS:</font>** Move the sliders segment out portions and find the best values that segment different components of the image.

In [ ]:
from ipywidgets import interactive,fixed

def color_threshold(im, rmin=-1,rmax=256,gmin=-1,gmax=256,bmin=-1,bmax=256):
    # Pull out the red, gree and blue matrixes
    r = im[:,:,0];
    g = im[:,:,1];
    b = im[:,:,2];
    b_img = ((r > rmin) & (r < rmax) & 
             (g > gmin) & (g < gmax) & 
             (b > bmin) & (b < bmax));
    plt.figure(figsize = (10,20));
    plt.imshow(b_img, cmap='gray', vmin=0, vmax=1);
    plt.show();
    return b_img*1;

w = interactive(color_threshold, im=fixed(im),
         rmin=(-1,256), rmax=(-1,256), 
         gmin=(-1,256), gmax=(-1,256), 
         bmin=(-1,256), bmax=(-1,256),__manual=True);
w

## The HSV colorspace

There are many alternatives to the RGB colorspace which can represent the exact same values.  For example, the HSV colorspace treats the color as a single value (Hue) and separates the intensity in the second and third channels.  In many applications this can make color based segmentation of images much easier. 


<img src="https://i.stack.imgur.com/HBq9I.png" width="35%">


Image from: https://dsp.stackexchange.com/questions/30238/what-is-the-difference-between-the-terms-color-intensity-and-color-saturation



In [ ]:
import colorsys
import matplotlib.colors as colors


hsv = colors.rgb_to_hsv(im)
f, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4,figsize=(20,5))
ax1.imshow(hsv[:,:,0], cmap='hsv')
ax1.set_title('Hue - Color')
ax1.set_axis_off()

ax2.imshow(hsv[:,:,1],cmap='gray',vmin=0, vmax=1)
ax2.set_title('Saturation - Amount of Color')
ax2.set_axis_off()

ax3.imshow(hsv[:,:,2],cmap='gray')
ax3.set_title('Value - Brightness')
ax3.set_axis_off()

ax4.imshow(im)
ax4.set_axis_off()

The Hue channel by itself is often enough to separate pixels with different colors. 

In [ ]:
plt.figure(figsize=(10,5));
plt.imshow(hsv[:,:,0], cmap='hsv');
plt.colorbar();

In [ ]:
from ipywidgets import interactive,fixed

def hsv_threshold(im, hmin=-0.01,hmax=1.01):
    # Pull out the red, gree and blue matrixes
    hsv = colors.rgb_to_hsv(im)
    h = hsv[:,:,0];
    
    # trick because the color space wraps
    if hmin > hmax:
        b_img = (h > hmin) | (h < hmax)
    else:
        b_img = (h > hmin) & (h < hmax);
    
    f, (ax1, ax3) = plt.subplots(1, 2,figsize=(20,5))
    ax1.imshow(im)
    ax1.set_axis_off()
    
    ax3.imshow(b_img,cmap='gray', vmin=0, vmax=1)
    ax3.set_title('Value - Brightness')
    ax3.set_axis_off()

    plt.show()
    return b_img*1;

w = interactive(hsv_threshold, im=fixed(im), hmin=(-0.01,1.01,0.01), hmax=(-0.01,1.01,0.01),__manual=True);
w

Sometimes using all three channels in HSV can make segmentation even easier. 

In [ ]:
from ipywidgets import interactive,fixed

def hsv_color_threshold(im, hmin=-0.01,hmax=1.01, smin=-0.01,smax=1.01,vmin=-1,vmax=256):
    # Pull out the red, gree and blue matrixes
    hsv = colors.rgb_to_hsv(im)
    h = hsv[:,:,0];
    s = hsv[:,:,1];
    v = hsv[:,:,2];
    
    # trick because the color space wraps
    if hmin > hmax:
        b_img = (h > hmin) | (h < hmax)
    else:
        b_img = (h > hmin) & (h < hmax);
    
    
    b_img = (b_img & 
         (s > smin) & (s < smax) & 
         (v > vmin) & (v < vmax));
    
    f, (ax1, ax2) = plt.subplots(1, 2,figsize=(20,5))
    ax1.imshow(im)
    ax1.set_axis_off()
    
    ax2.imshow(b_img,cmap='gray', vmin=0, vmax=1)
    ax2.set_title('Value - Brightness')
    ax2.set_axis_off()

    plt.show()
    return b_img*1;

w = interactive(hsv_color_threshold, im=fixed(im),
         hmin=(-0.01,1.01,0.01), hmax=(-0.01,1.01,0.01), 
         smin=(-0.01,1.01,0.01), smax=(-0.01,1.01,0.01), 
         vmin=(-1,256), vmax=(-1,256),__manual=True);
w

----
<a name="Practice_Example"></a>

# 2. Practice Example

&#9989; **<font color=red>DO THIS:</font>** Consider the following example image.  Based on what you have learned segment the image to only include the contents of the petri dish.  Feel free to work with your neighbor and try to figure out the following parts:

- Find color thresholds to generate a binary image with just the dish
- Create a three channel image that removes the background (sets it to black) and only keeps the dish. 

Be prepared to share your answer with the class. If you get the dish segmented see if you can then independently segment out the two different bacterial colonies found in the image. 

In [ ]:
url = 'https://goo.gl/cebwc1'
file1 = 'Petri_dish1.jpg'
urlretrieve(url, file1);

im=imread(file1)
plt.imshow(im)

In [ ]:
# Put your segmentation code here.

&#9989; **<font color=red>QUESTION:</font>** Could we use machine learning to solve this problem?  What would our training and testing data look like?  

Put your answer to the above question here.

----
Written by Dr. Dirk Colbry, Michigan State University
<a rel="license" href="http://creativecommons.org/licenses/by-nc/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc/4.0/88x31.png" /></a><br />This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc/4.0/">Creative Commons Attribution-NonCommercial 4.0 International License</a>.

----